# Introdução ao Python

In [ ]:
print("Olá, Mundo")

In [ ]:
print("Bem-vindos ao minicurso de python aplicado a meteorologia")

In [ ]:
print(2 + 2)

In [ ]:
print(50 - 5*6)

In [ ]:
print(8 / 5)

In [ ]:
a = 20
b = 5 * 9
c = a + b
print(c)

In [ ]:
listas = [1, 4, 9, 16, 25]
print(listas)

In [ ]:
for numero in range(1, 10):
  print(numero)


In [ ]:
temperatura = 20

if temperatura < 50:
  print("Temperatura menor que 50 graus Celsius")

In [ ]:
temperatura = 45

if temperatura < 40:
  print("Temperatura menor que 40 graus Celsius")
else:
  print("Temperatura maior que 40 graus Celsius")


In [ ]:
umidade_relativa = 80
temperatura = 20

if umidade_relativa <= 30 and temperatura >= 40:
  print("Condição perigosa")
else:
  print("Condição favorável")

In [ ]:
umidade_relativa = 80
temperatura = 45

if umidade_relativa <= 30 or temperatura >= 40:
  print("Condição perigosa")
else:
  print("Condição favorável")

# Vamos a prática

## Carregando bibliotecas

In [ ]:
!pip install netCDF4
!pip install cfgrib
!pip install windrose

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from windrose import WindroseAxes
import seaborn as sns

## Importando dados

In [ ]:
#################################################################
# IMPORTANDO DADOS DO ERA5 --> Registros de vento, pressão atmosférica, temperatura do ar e temperatura do ponto de orvalho
#################################################################
ds_ERA5 = xr.open_dataset('/content/ERA5_2015-2024.grib', engine='cfgrib')
print(ds_ERA5)

In [ ]:
###################################################
# IMPORTANDO DADOS DO MERGE-CPTEC --> Registros de Chuva
###################################################
ds_MERGE = xr.open_dataset('/content/MERGE_CPTEC_BRASIL_2015-2024.nc')
print(ds_MERGE)

In [ ]:
# Calculando a Velocidade do Vento a 10metros
ds_ERA5['vv10m'] = np.sqrt(ds_ERA5['v10']**2 + ds_ERA5['u10']**2)

# Calculando a direção do vento
ds_ERA5['dv10m'] = (np.rad2deg(np.arctan2(-ds_ERA5['u10'], -ds_ERA5['v10'])) + 360) % 360

# Conversão da temperatura do ar de Kelvin para Celsius
ds_ERA5['temp_c'] = ds_ERA5['t2m'] - 273.15

# Conversão da temperatura do ponto de orvalho de Kelvin para Celsius
ds_ERA5['temp_orvalho_c'] = ds_ERA5['d2m'] - 273.15

# Conversão da pressão atmosférica de Pa para hPa
ds_ERA5['pressao_atm'] = ds_ERA5['sp']/100

# Calculando a pressão de vapor de saturação e pressão de vapor real
T = ds_ERA5['temp_c']
Td = ds_ERA5['temp_orvalho_c']

es = 6.112 * np.exp((17.67 * T) / (T + 243.5))
e = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))

# Calcula a Umidade Relativa
ds_ERA5['umidade_rel'] = 100 * (e / es)

## Séries temporais

In [ ]:
lat = -9.55   # sua latitude
lon = -35.77   # sua longitude

# Seleciona o ponto mais próximo dessa coordenada
ERA5_ponto = ds_ERA5.sel(latitude=lat, longitude=lon, method='nearest')
MERGE_ponto = ds_MERGE.sel(lat=lat, lon=lon, method='nearest')

In [ ]:
# Extraindo a série temporal
serie = ERA5_ponto['temp_c']

# Plotando
plt.figure(figsize=(10, 4))
serie.plot()
plt.title(f"Temperatura do Ar em Maceió/Alagoas")
plt.xlabel("Data")
plt.ylabel("[°C]")  # ou outra unidade conforme a variável
plt.grid()
plt.show()

In [ ]:
# Extraindo a série temporal
serie = ERA5_ponto['temp_c']

# Plotando com personalizações
plt.figure(figsize=(12, 5))
serie.plot(color='blue', linestyle='-', linewidth=2, marker='o', markersize=4)

plt.title("Maceió/Alagoas", fontsize=14, fontweight='bold')
plt.xlabel("Data", fontsize=12)
plt.ylabel("Temperatura do Ar [°C]", fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.9)
plt.ylim(23,29)
plt.tight_layout()
plt.show()

In [ ]:
# Extraindo as séries
serie1 = ERA5_ponto['temp_c']         # Temperatura do Ar [°C]
serie2 = ERA5_ponto['umidade_rel']    # Umidade Relativa [%]

# ========================
# CRIAÇÃO DA FIGURA E EIXO PRINCIPAL (TEMPERATURA)
# ========================
fig, ax1 = plt.subplots(figsize=(12, 5))

# Plotando a temperatura
ax1.plot(serie1['time'], serie1.values, color='blue', linestyle='-', linewidth=2, marker='o', markersize=4)
ax1.set_xlabel("Data", fontsize=12)
ax1.set_ylabel("Temperatura do Ar [°C]", fontsize=12, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_ylim(23, 29)

# Criando o segundo eixo y
ax2 = ax1.twinx()

# ========================
# PRIMEIRO EIXO SECUNDÁRIO (UMIDADE RELATIVA)
# ========================
ax2.plot(serie2['time'], serie2.values, color='green', linestyle='--', linewidth=2, marker='s', markersize=4)
ax2.set_ylabel("Umidade Relativa [%]", fontsize=12, color='green')
ax2.tick_params(axis='y', labelcolor='green')
ax2.set_ylim(60, 100)  # Ajuste conforme os valores reais da variável

# Título
plt.title("Maceió/Alagoas", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle='--', alpha=0.5)

# Layout
fig.tight_layout()
plt.show()

In [ ]:
# Extraindo as variáveis
barra = MERGE_ponto['pacum']             # Precipitação
linha1 = ERA5_ponto['temp_c']            # Temperatura
linha2 = ERA5_ponto['umidade_rel']       # Umidade Relativa

# ========================
# CRIAÇÃO DA FIGURA E EIXO PRINCIPAL (PRECIPITÇÃO)
# ========================
fig, ax1 = plt.subplots(figsize=(13, 5))

# Variável em barras (precipitação)
dates = mdates.date2num(barra['time'])
ax1.bar(dates, barra.values, color='blue', width=20)
ax1.xaxis_date()  # para manter o formato de data no eixo X
ax1.set_xlabel("Data", fontsize=12)
ax1.set_ylabel("Precipitação [mm]", fontsize=12, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_ylim(0, 500)

# ========================
# PRIMEIRO EIXO SECUNDÁRIO (TEMPERATURA)
# ========================
ax2 = ax1.twinx()
ax2.plot(linha1['time'], linha1.values, color='orangered', linestyle='-', linewidth=2, marker='o')
ax2.set_ylabel("Temperatura [°C]", fontsize=12, color='orangered')
ax2.tick_params(axis='y', labelcolor='orangered')
ax2.set_ylim(23, 29)

# ========================
# TERCEIRO EIXO (UMIDADE RELATIVA)
# ========================
ax3 = ax1.twinx()
ax3.spines.right.set_position(("axes", 1.1))  # desloca o eixo para a direita (para não sobrepor o segundo)
ax3.plot(linha2['time'], linha2.values, color='green', linestyle='--', linewidth=2, marker='s')
ax3.set_ylabel("Umidade Relativa [%]", fontsize=12, color='green')
ax3.tick_params(axis='y', labelcolor='green')
ax3.set_ylim(60, 100)

# Título
plt.title("Maceió/AL", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle='--', alpha=0.5)

fig.tight_layout()
plt.show()

In [ ]:
# Filtro para dados
inicio = '2020-01-01'
fim = '2020-12-31'

# Extraindo as variáveis no intervalo definido
barra = MERGE_ponto['pacum'].sel(time=slice(inicio, fim))          # Precipitação
linha1 = ERA5_ponto['temp_c'].sel(time=slice(inicio, fim))         # Temperatura
linha2 = ERA5_ponto['umidade_rel'].sel(time=slice(inicio, fim))    # Umidade Relativa

# ========================
# CRIAÇÃO DA FIGURA E EIXO PRINCIPAL (PRECIPITAÇÃO)
# ========================
fig, ax1 = plt.subplots(figsize=(13, 5))

# Variável em barras (precipitação)
dates = mdates.date2num(barra['time'])
ax1.bar(dates, barra.values, color='blue', width=20)
ax1.xaxis_date()
ax1.set_xlabel("Data", fontsize=12)
ax1.set_ylabel("Precipitação [mm]", fontsize=12, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_ylim(0, 500)
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b')) # Formatando o eixo X para mostrar os meses abreviados

# ========================
# PRIMEIRO EIXO SECUNDÁRIO (TEMPERATURA)
# ========================
ax2 = ax1.twinx()
ax2.plot(linha1['time'], linha1.values, color='orangered', linestyle='-', linewidth=2, marker='o')
ax2.set_ylabel("Temperatura [°C]", fontsize=12, color='orangered')
ax2.tick_params(axis='y', labelcolor='orangered')
ax2.set_ylim(23, 29)

# ========================
# TERCEIRO EIXO (UMIDADE RELATIVA)
# ========================
ax3 = ax1.twinx()
ax3.spines.right.set_position(("axes", 1.1))  # desloca o eixo para a direita (para não sobrepor o segundo)
ax3.plot(linha2['time'], linha2.values, color='green', linestyle='--', linewidth=2, marker='s')
ax3.set_ylabel("Umidade Relativa [%]", fontsize=12, color='green')
ax3.tick_params(axis='y', labelcolor='green')
ax3.set_ylim(60, 100)

# Título
plt.title("Maceió/AL - 2020", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle='--', alpha=0.5)

fig.tight_layout()
plt.show()

In [ ]:
# Criando a rosa dos ventos
wind_speed = ERA5_ponto['vv10m'].values
wind_dir = ERA5_ponto['dv10m'].values

# Criar a rosa dos ventos
ax = WindroseAxes.from_ax()

# Plotando os dados com cores definidas
ax.bar(wind_dir, wind_speed, normed=True, opening=0.9, edgecolor='white')
ax.set_theta_zero_location("E")  # Define a direção do vento como referência
ax.set_xticklabels(["E", "NE", "N", "NW", "W", "SW", "S", "SE"], fontsize=15)

# Adiciona legenda com tamanho de fonte
ax.set_legend(fontsize=20, bbox_to_anchor=(-0.3, 0.3), title="(m/s)")

# Título do gráfico
plt.title('Direção predominante e velocidade do vento - Maceió/AL', fontsize=20, y=1.08)
plt.show()

In [ ]:
# Extraindo a série temporal da variável de interesse (temperatura do ar em ºC)
# Converte para DataFrame e reseta o índice para facilitar a manipulação
serie = ERA5_ponto['temp_c'].to_dataframe().reset_index()

# Extraindo o ano e o mês da coluna de tempo
serie['Ano'] = serie['time'].dt.year
serie['Mês'] = serie['time'].dt.month

# Lista com os nomes dos meses em português
Mes = [
    "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
    "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"
]

# Mapeando os números dos meses (1-12) para seus respectivos nomes
serie['Mês'] = serie['Mês'].apply(lambda x: Mes[x - 1])

# Convertendo a coluna de meses para uma categoria ordenada (garante a ordem correta no gráfico)
serie['Mês'] = pd.Categorical(serie['Mês'], categories=Mes, ordered=True)

# Reorganizando o DataFrame em formato de tabela (estilo matriz)
# Índice = Mês, Colunas = Ano, Valores = Temperatura
dados = serie.pivot(index="Mês", columns="Ano", values="temp_c")

# Criando a figura e os eixos (um para o gráfico e outro para a barra de cores)
f, (ax, cbar_ax) = plt.subplots(
    2,                             # Dois eixos: um para o heatmap e outro para a barra de cores
    gridspec_kw={
        "height_ratios": (0.9, 0.05),  # Tamanho relativo dos eixos
        "hspace": 0.3                  # Espaço entre os eixos
    },
    figsize=(9, 6)                # Tamanho da figura
)

# Criando o heatmap com Seaborn
ax = sns.heatmap(
    dados,                        # Dados organizados por mês e ano
    ax=ax,                        # Eixo principal
    annot=True,                   # Exibe os valores dentro das células
    square=False,                 # Células não precisam ser quadradas
    annot_kws={"size": 10},       # Tamanho do texto dentro das células
    yticklabels=Mes,              # Rótulos do eixo Y (nomes dos meses)
    cbar_ax=cbar_ax,              # Eixo da barra de cores
    cbar_kws={"orientation": "horizontal"},  # Barra de cores na horizontal
    cmap="jet",                   # Paleta de cores
    vmin=23,                      # Valor mínimo da escala de cor
    vmax=28,                      # Valor máximo da escala de cor
    linewidths=0.5                # Espessura das linhas entre as células
)

# Definindo o título do gráfico
ax.set_title("Heatmap da Temperatura do Ar em Maceió/AL", fontsize=12, y=1.08)

# Personalizando os eixos
ax.tick_params(labelsize=12)     # Tamanho dos rótulos do eixo
ax.set_xlabel("Ano", fontsize=12)
ax.set_ylabel("Mês", fontsize=12)

# Ajustando a fonte da barra de cores
cbar_ax.tick_params(labelsize=12)

# Links Úteis

Seleção de cores:

https://matplotlib.org/stable/users/explain/colors/colormaps.html

https://matplotlib.org/stable/gallery/color/named_colors.html

Estilos de linha:

https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html

Estilo de marcadores:

https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html


Espessura da linha:

https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html